In [4]:
%serialconnect to --port COM9

Connecting to --port=COM9 --baud=115200 
Ready.


[參考網頁](http://yhhuang1966.blogspot.com/2017/06/micropython-on-esp8266-dht11.html)
## 如果要在手機上觀看 ThingSpeak 資料, 可以到 Google Play 下載 ThingView 這個 App

## 修改 boot.py，開機時連接無線網路，並取得 IP

In [5]:
%sendtofile boot.py
try:
  import usocket as socket
except:
  import socket

import network
from machine import Pin
import dht

import esp
esp.osdebug(None)

import gc
gc.collect()

ssid = 'informatics'
password = '0953313123'

station = network.WLAN(network.STA_IF)

station.active(True)
station.connect(ssid, password)

while station.isconnected() == False:
  pass

print('Connection successful')
print(station.ifconfig())

#sensor = dht.DHT22(Pin(14))
sensor = dht.DHT11(Pin(5))

Sent 31 lines (461 bytes) to boot.py.


## 測試 1 : 連續測量溫溼度 

In [ ]:
#%sendtofile main.py
from machine import Pin
import dht     
import time

p0=Pin(5, Pin.IN)
d=dht.DHT11(p0)        #建立 DHT11 物件

while True:
    d.measure()                  #重新測量溫溼度
    t=d.temperature()        #讀取攝氏溫度
    h=d.humidity()             #讀取相對溼度
    print('Temperature=', t, 'C', 'Humidity=', h, '%')
    time.sleep(2)                  #暫停 2 秒


Temperature= 28 C Humidity= 41 %
Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 41 %
Temperature= 28 C Humidity= 41 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 42 %
.Temperature= 28 C Humidity= 43 %
Temperature= 28 C Humidity= 43 %
.Temperature= 28 C Humidity= 42 %
Temperature= 28 C Humidity= 43 %
Temperature= 28 C Humidity= 43 %
.Temperature= 28 C Humidity= 43 %
Temperature= 28 C Humidity= 43 %

## 測試 2 : 將溫濕度記錄在 ThingSpeak 網站

In [3]:
%sendtofile main.py
#main.py
ffrom machine import Pin
import dht   
import time
import urequests 

p0=Pin(5, Pin.IN)
d=dht.DHT11(p0)
host='http://api.thingspeak.com'       
read_api_key='RNAXB36EELSPQKXN'

while True:
    d.measure()                 
    t=d.temperature()       
    f=round(t * 9/5 + 32) 
    h=d.humidity()
    url='%s/update?api_key=%s&field1=%s&field2=%s&field3=%s' %(host, api_key, t, f, h)   
    print('Temperature=', t, 'C', '/', f, 'F', 'Humidity=', h, '%')
    #print('url=', url)   
    r=urequests.get(url)   
    print('response=', r.text)
    time.sleep(16)   

Sent 22 lines (571 bytes) to main.py.


In [ ]:
#%sendtofile myapp.py
#myapp.py
import urequests
import time

host='http://api.thingspeak.com'
read_api_key='RNAXB36EELSPQKXN'
channel_id='368523'

url='%s/channels/%s/feeds/last.json?api_key=%s' \
     %(host, channel_id, read_api_key)

while True:
    try:
        r=urequests.get(url)
        print(r.text) 
        print(r.json()) 
    except:
        print('urequests.get() exception occurred!')
    time.sleep(3)


.urequests.get() exception occurred!
..urequests.get() exception occurred!
..urequests.get() exception occurred!
..urequests.get() exception occurred!
.

In [6]:
%sendtofile myapp.py
#myapp.py
import urequests
import time

host='http://api.thingspeak.com'
read_api_key='RNAXB36EELSPQKXN'
channel_id='368523'

url='%s/channels/%s/feeds/last.json?api_key=%s' \
     %(host, channel_id, read_api_key)

while True:
    try:
        r=urequests.get(url)
        print(r.text) 
        print(r.json()) 
    except:
        print('urequests.get() exception occurred!')
    time.sleep(3)


url='%s/channels/%s/feeds/last.json?api_key=%s' \
     %(host, channel_id, read_api_key)

while True:
    try:
        r=urequests.get(url)
        d=r.json() 
        print(d['created_at'],d['entry_id'],d['field1'],d['field2'],
              d['field3'],d['field4'],d['field5'])
    except:
        print('urequests.get() exception occurred!')
    time.sleep(10)

Sent 33 lines (762 bytes) to myapp.py.


## 測試 3 : 等待 ThingSpeak 更新週期時閃爍 LED

In [6]:
%sendtofile main.py
from machine import Pin
import dht   
import time
import urequests

p0=Pin(5, Pin.IN)
p2=Pin(14, Pin.OUT)     
d=dht.DHT11(p0)
host='http://api.thingspeak.com'
api_key='RNAXB36EELSPQKXN'

def LED_blink(s):    
    for i in range(1,10*s):    
        p2.value(1)   
        time.sleep_ms(50)        
        p2.value(0)
        time.sleep_ms(50)    

while True:
    d.measure()                 
    t=d.temperature()       
    f=round(t * 9/5 + 32) 
    h=d.humidity()
    url='%s/update?api_key=%s&field1=%s&field2=%s&field3=%s' %(host, api_key, t, f, h)
    print('Temperature=', t, 'C', '/', f, 'F', 'Humidity=', h, '%')
    try:
        r=urequests.get(url)
        print('response=', r.text)
    except:
        print('urequests.get() exception occurred!')
    LED_blink(16) 

Sent 31 lines (781 bytes) to main.py.


![DHT11](images\\DHT11.jpg)
![接線圖](images\\MP_DHT11_bb.jpg)